In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import cv2
import json
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
BASE_DIR = '../input/cassava-leaf-disease-classification'

In [ ]:
df = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))

# Training Parameters - Epochs, Image-Size, Batch-Size

In [ ]:
BATCH_SIZE = 24
EPOCHS = 25
TARGET_SIZE = 200

# Data Augmentation

In [ ]:
df.label = df.label.astype('str')

train_datagen = ImageDataGenerator(featurewise_center = True,
                                   featurewise_std_normalization = True,
                                   validation_split = 0.25,
                                   rotation_range = 20,
                                   zoom_range = 0.25,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest',
                                   height_shift_range = 0.1,
                                   width_shift_range = 0.1)


train_generator = train_datagen.flow_from_dataframe(df,
                         directory = os.path.join(BASE_DIR, "train_images"),
                         subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

validation_datagen = ImageDataGenerator(validation_split = 0.25)

validation_generator = validation_datagen.flow_from_dataframe(df,
                         directory = os.path.join(BASE_DIR, "train_images"),
                         subset = "validation",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

# Visualize few augmentations

In [ ]:
generator = train_datagen.flow_from_dataframe(df.iloc[21:22],
                         directory = os.path.join(BASE_DIR, "train_images"),
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

aug_images = [generator[0][0][0]/255 for i in range(10)]
fig, axes = plt.subplots(2, 5, figsize = (20, 10))
axes = axes.flatten()
for img, ax in zip(aug_images, axes):
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout()
plt.show()

# Creating the Model

In [ ]:
def create_model():
    conv_base = EfficientNetB7(include_top = False, weights = 'imagenet',
                               input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dense(512, activation = 'relu')(model)
    model = layers.Dropout(0.5)(model)
    model = layers.Dense(5, activation = "softmax")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.001), loss = "sparse_categorical_crossentropy", metrics = ["acc"])
    return model

In [ ]:
model = create_model()
model.summary()

# Training the model with callbacks - Checkpoint, EarlyStopping & Reduce_lr

In [ ]:
checkpoint = ModelCheckpoint('EfficientNetB7-Im200.h5', save_best_only = True, monitor = 'val_loss', mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, patience = 10, mode = 'min', verbose = 1, restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, patience = 3, min_delta = 0.001, mode = 'min', verbose = 1)


history = model.fit(train_generator, epochs = EPOCHS, validation_data = validation_generator, validation_steps = 200, callbacks = [checkpoint, early_stop, reduce_lr])

# Model Performance Graphs

## Accuracy Graph

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Loss Graph

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Visualizing the GradCam at the intermediate Layers

In [ ]:
def visualize_activation(image, activation_layer = 0, layers = 10):
    
    layer_outputs = [layer.output for layer in model.layers[:layers]]
    activation_model = models.Model(inputs = model.input, outputs = layer_outputs)
    activations = activation_model.predict(image)
    
    rows = int(activations[activation_layer].shape[3] / 3)
    cols = int(activations[activation_layer].shape[3] / rows)
    
    fig, axes = plt.subplots(rows, cols, figsize = (15, 15 * cols))
    axes = axes.flatten()
    
    for i, ax in zip(range(activations[activation_layer].shape[3]), axes):
        
        ax.matshow(activations[activation_layer][0, :, :, i], cmap = 'viridis')
        ax.axis('off')
        
    plt.tight_layout()
    plt.show()

In [ ]:
img_path = os.path.join(BASE_DIR, "train_images", df.image_id[40])
img = image.load_img(img_path, target_size = (TARGET_SIZE, TARGET_SIZE))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis = 0)
img_tensor /= 255.

In [ ]:
# 0th Layer
visualize_activation(img_tensor, 0)

In [ ]:
# 10th Layer
visualize_activation(img_tensor, 4)

In [ ]:
def all_activations_vis(img, layers = 10):
    layer_outputs = [layer.output for layer in model.layers[:layers]]
    activation_model = models.Model(inputs = model.input, outputs = layer_outputs)
    activations = activation_model.predict(img)
    
    layer_names = []
    for layer in model.layers[:layers]: 
        layer_names.append(layer.name) 

    images_per_row = 3
    for layer_name, layer_activation in zip(layer_names, activations): 
        n_features = layer_activation.shape[-1] 

        size = layer_activation.shape[1] 

        n_cols = n_features // images_per_row 
        display_grid = np.zeros((size * n_cols, images_per_row * size)) 

        for col in range(n_cols): 
            for row in range(images_per_row): 
                channel_image = layer_activation[0, :, :, col * images_per_row + row] 
                channel_image -= channel_image.mean() 
                channel_image /= channel_image.std() 
                channel_image *= 64 
                channel_image += 128 
                channel_image = np.clip(channel_image, 0, 255).astype('uint8') 
                display_grid[col * size : (col + 1) * size, 
                             row * size : (row + 1) * size] = channel_image 
        scale = 1. / size 
        plt.figure(figsize=(scale * 5 * display_grid.shape[1], 
                            scale * 5 * display_grid.shape[0])) 
        plt.title(layer_name) 
        plt.grid(False)
        plt.axis('off')
        plt.imshow(display_grid, aspect = 'auto', cmap = 'viridis')

In [ ]:
all_activations_vis(img_tensor, 5)

# Submission 

## Opening the Submission dataframe

In [ ]:
submit = pd.read_csv(os.path.join(BASE_DIR, "sample_submission.csv"))

## Running Predictions

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

preds = []

for image_id in submit.image_id:
    
    image = load_img(os.path.join(BASE_DIR, "test_images", image_id), target_size = (TARGET_SIZE, TARGET_SIZE))
    
    #Image Processes & Reshaping
    image = img_to_array(image)    
    image = image.reshape(1, TARGET_SIZE, TARGET_SIZE, 3)

    #Center Pixel Data & Normalize into float
    image = image.astype('float32')
    preds.append(np.argmax(model.predict(image)))

submit['label'] = preds

## Exporting the submission.csv file

In [ ]:
submit.to_csv('submission.csv', index = False)